In [ ]:
# Import libraries
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import sum, col, when, max, lag
from snowflake.snowpark import Window
from datetime import timedelta
import altair as alt
import streamlit as st
import pandas as pd



In [ ]:
session.table("FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES").sample(0.5)


In [ ]:
session = get_active_session()
snow_df_stocks = (
        session.table("FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES")
        .filter(
            (col('TICKER').isin('AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA')) & 
            (col('VARIABLE_NAME').isin('Nasdaq Volume', 'Post-Market Close')))
        .groupBy("TICKER", "DATE")
        .agg(
            max(when(col("VARIABLE_NAME") == "Nasdaq Volume", col("VALUE"))).alias("NASDAQ_VOLUME"),
            max(when(col("VARIABLE_NAME") == "Post-Market Close", col("VALUE"))).alias("POSTMARKET_CLOSE")
        )
    )

In [ ]:
snow_df_stocks.sort(col('DATE').desc(), col('TICKER')).show()

In [ ]:
# Adding the Day over Day Post-market Close Change calculation
window_spec = Window.partitionBy("TICKER").orderBy("DATE")
snow_df_stocks_transformed = snow_df_stocks.withColumn("DAY_OVER_DAY_CHANGE", 
    (col("POSTMARKET_CLOSE") - lag(col("POSTMARKET_CLOSE"), 1).over(window_spec)) /
    lag(col("POSTMARKET_CLOSE"), 1).over(window_spec)
)

# Load foreign exchange (FX) rates data.
snow_df_fx = session.table("FINANCE__ECONOMICS.CYBERSYN.FX_RATES_TIMESERIES").filter(
    (col('BASE_CURRENCY_ID') == 'EUR') & (col('DATE') >= '2019-01-01')).with_column_renamed('VARIABLE_NAME','EXCHANGE_RATE')


In [ ]:
snow_df_stocks_transformed.show(10)

In [ ]:
snow_df_stocks.with_columns(["LAG_POSTMARKET_CLOSE", "DAY_OVER_DAY_CHANGE"],
    [lag(col("POSTMARKET_CLOSE"), 1).over(window_spec), (col("POSTMARKET_CLOSE") - col("LAG_POSTMARKET_CLOSE")) / 
     col("LAG_POSTMARKET_CLOSE")])